<a href="https://colab.research.google.com/github/veryHapppy/study_ai/blob/main/Kaggle/dog_breed_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna
!pip install plotly
!pip install konlpy
!pip install transformers datasets accelerate
!pip install evaluate
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import torch.nn.functional as F
import optuna
import optuna.visualization as vis
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import itertools
from sklearn.metrics import confusion_matrix
from konlpy.tag import Okt
import urllib.request
import pickle
from collections import Counter
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
import evaluate
from transformers import pipeline
import os
from torch.utils.data import Dataset, DataLoader, random_split
import zipfile
import json
import torchvision.models as models
from torchvision.datasets import ImageFolder

from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 135.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
class DogBreedDataset(Dataset):
    def __init__(self, root_dir, csv_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.df = pd.read_csv(csv_file)

        # 견종 이름(문자열)을 숫자로 바꾸기 위한 딕셔너리
        self.breeds = sorted(self.df['breed'].unique())
        self.breed_to_idx = {breed: i for i, breed in enumerate(self.breeds)}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # CSV에서 이미지 ID와 견종 가져오기
        img_id = self.df.iloc[idx, 0]
        breed_name = self.df.iloc[idx, 1]

        # 이미지 경로 생성 (ID에 .jpg 붙이기)
        img_path = os.path.join(self.root_dir, f"{img_id}.jpg")
        image = Image.open(img_path).convert('RGB')

        # 라벨을 숫자로 변환 (0~119)
        label = self.breed_to_idx[breed_name]

        if self.transform:
            image = self.transform(image)

        return image, label


train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(), # 컬러 이미지에선 필수!
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# 2. 테스트/검증용 (증강 제외 - 깨끗한 원본)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# 2. 데이터셋 생성 (경로를 본인의 폴더명으로 바꾸세요)

zip_path = '/content/drive/MyDrive/Colab Notebooks/Dog Breed Identification/train.zip'
extract_path = '/content/dataset/train' # 코랩 내부 경로

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_path)

# 1. 전체 데이터셋 생성 (아까 만든 DogCatDataset 활용)
full_dataset = DogBreedDataset(root_dir='/content/dataset/train',csv_file='/content/drive/MyDrive/Colab Notebooks/Dog Breed Identification/labels.csv', transform=None)

# 2. 비율 정하기 (예: 80%는 학습용, 20%는 검증용)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

# 3. 무작위로 데이터 분할
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

class ApplyTransform(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

# 이제 각각의 로더에 맞게 적용
train_dataset = ApplyTransform(train_dataset, transform=train_transform)
val_dataset = ApplyTransform(val_dataset, transform=test_transform)

# 4. 각각의 DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False) # 검증은 섞을 필요 없음

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
def train_and_eval(model, train_loader, test_loader, optimizer, criterion):
    # 3 에폭 정도만 학습해서 성능을 확인 (시간 절약)
    for epoch in range(2):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # 최종 검증 단계
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

def objective_step1(trial):
    # 1. 하이퍼파라미터 제안
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)

    # 2. 데이터 로더 설정
    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                              shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # 3. 모델 초기화
    model = models.resnet50(weights='DEFAULT')
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 120)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    for param in model.parameters(): param.requires_grad = False
    for param in model.fc.parameters(): param.requires_grad = True

    optimizer_head = optim.Adam(model.fc.parameters(), lr=1e-3)

    model.train()
    for i, (images, labels) in enumerate(train_loader):
        if i > 50:
            break # 약 1,600장(50배치) 정도만 학습하고 넘어감
        images, labels = images.to(device), labels.to(device)
        optimizer_head.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_head.step()

    for param in model.parameters(): param.requires_grad = True

    optimizer_full = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    acc = train_and_eval(model, train_loader, val_loader, optimizer_full, criterion)
    return acc

# 1단계 실행
study1 = optuna.create_study(direction="maximize")
study1.optimize(objective_step1, n_trials=20) # 시도횟수

best_lr = study1.best_params['lr']
best_batch_size = study1.best_params['batch_size']
best_weight_decay = study1.best_params['weight_decay']
print(f"1단계 완료, 최적의 lr: {best_lr}, batch_size: {best_batch_size}, weight_decay: {best_weight_decay}")

[I 2026-01-02 04:30:05,701] A new study created in memory with name: no-name-0bf9143f-8799-4f3a-819c-7da98ce816bd


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 164MB/s]
[I 2026-01-02 04:33:13,690] Trial 0 finished with value: 84.05867970660147 and parameters: {'batch_size': 32, 'lr': 2.1752200442071606e-05, 'weight_decay': 1.408523350365764e-06}. Best is trial 0 with value: 84.05867970660147.
[I 2026-01-02 04:36:34,300] Trial 1 finished with value: 60.440097799511 and parameters: {'batch_size': 64, 'lr': 0.0007201066770823708, 'weight_decay': 0.00022275982569817257}. Best is trial 0 with value: 84.05867970660147.
[I 2026-01-02 04:39:51,619] Trial 2 finished with value: 63.960880195599024 and parameters: {'batch_size': 64, 'lr': 0.0007069144461813384, 'weight_decay': 0.00021433779506590162}. Best is trial 0 with value: 84.05867970660147.
[I 2026-01-02 04:43:05,244] Trial 3 finished with value: 48.36185819070904 and parameters: {'batch_size': 16, 'lr': 0.000762780378271271, 'weight_decay': 6.769882095301555e-06}. Best is trial 0 with value: 84.05867970660147.
[I 2026-01-02 04:46:18,843] Trial 4 finishe

1단계 완료, 최적의 lr: 4.0122966671348226e-05, batch_size: 64, weight_decay: 2.8290255515133917e-06


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/Dog Breed Identification/best_params_step1.json', 'w') as f:
    json.dump(study1.best_params, f)

In [ ]:
# 2. 그 레시피대로 '진짜 최종 모델'을 생성합니다.
final_model = models.resnet50(weights='DEFAULT')
num_ftrs = final_model.fc.in_features
final_model.fc = nn.Linear(num_ftrs, 120) # 120
final_model = final_model.to(device)

history = {'train_loss': [], 'val_acc': []}

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_dataset = ApplyTransform(train_dataset, transform=train_transform)
val_dataset = ApplyTransform(val_dataset, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=best_batch_size,
                          shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


for param in final_model.parameters(): param.requires_grad = False
for param in final_model.fc.parameters(): param.requires_grad = True

optimizer = optim.Adam(final_model.fc.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
print("Starting Phase 1...")
for epoch in range(5):
    final_model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = final_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Phase 1 - Epoch [{epoch+1}/5] Loss: {total_loss/len(train_loader):.4f}")

for param in final_model.parameters(): param.requires_grad = True

optimizer = optim.Adam(final_model.parameters(), lr=best_lr, weight_decay=best_weight_decay)
# 학습률 스케줄러 추가 (성능 향상 정체 시 0.1배 감소)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3)
epochs = 30
for epoch in range(epochs):
    final_model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = final_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # 에폭마다 검증 (테스트셋 활용)
    final_model.eval()
    correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = final_model(images)
            _, pred = torch.max(outputs, 1)
            correct += (pred == labels).sum().item()
    acc = 100 * correct / len(val_dataset)
    avg_loss = total_loss / len(train_loader)

    scheduler.step(acc)
    history['train_loss'].append(avg_loss)
    history['val_acc'].append(acc)

    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {avg_loss:.4f}, Acc: {acc:.2f}%")

print("최종 모델 학습 완료")

Starting Phase 1...
Phase 1 - Epoch [1/5] Loss: 3.3058
Phase 1 - Epoch [2/5] Loss: 1.7367
Phase 1 - Epoch [3/5] Loss: 1.3674
Phase 1 - Epoch [4/5] Loss: 1.2317
Phase 1 - Epoch [5/5] Loss: 1.1601
Epoch [1/30] - Loss: 1.0696, Acc: 87.24%
Epoch [2/30] - Loss: 0.9972, Acc: 87.53%
Epoch [3/30] - Loss: 0.9601, Acc: 87.78%
Epoch [4/30] - Loss: 0.9330, Acc: 87.58%
Epoch [5/30] - Loss: 0.9131, Acc: 87.29%
Epoch [6/30] - Loss: 0.8973, Acc: 87.09%
Epoch [7/30] - Loss: 0.8843, Acc: 87.19%
Epoch [8/30] - Loss: 0.8744, Acc: 87.48%
Epoch [9/30] - Loss: 0.8703, Acc: 87.33%
Epoch [10/30] - Loss: 0.8706, Acc: 87.58%
Epoch [11/30] - Loss: 0.8673, Acc: 87.43%
Epoch [12/30] - Loss: 0.8670, Acc: 87.48%
Epoch [13/30] - Loss: 0.8660, Acc: 87.53%
Epoch [14/30] - Loss: 0.8654, Acc: 87.48%
Epoch [15/30] - Loss: 0.8654, Acc: 87.09%
Epoch [16/30] - Loss: 0.8659, Acc: 87.14%
Epoch [17/30] - Loss: 0.8657, Acc: 87.43%
Epoch [18/30] - Loss: 0.8667, Acc: 87.48%
Epoch [19/30] - Loss: 0.8667, Acc: 87.29%
Epoch [20/30] - 

In [ ]:
save_path = '/content/drive/MyDrive/Colab Notebooks/Dog Breed Identification'

if not os.path.exists(save_path):
    os.makedirs(save_path)

# 모델 저장 (가장 성능이 좋았을 때 실행)
torch.save(final_model.state_dict(), f"{save_path}/final_model.pth")

In [ ]:
class PlantTestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        # test 폴더 내의 모든 이미지 파일 목록 (정렬하여 순서 보장)
        self.file_list = sorted([f for f in os.listdir(root_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_name = self.file_list[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_name # 이미지와 파일명을 함께 반환

def generate_submission(model, test_loader, class_names, save_path):
    model.eval()
    all_probs = []
    all_files = []

    with torch.no_grad():
        for images, file_names in test_loader:
            images = images.to(device)
            outputs = model(images)

            # 1. 각 클래스별 확률값 계산 (Softmax)
            probs = F.softmax(outputs, dim=1)

            # CPU로 옮기고 리스트에 저장
            all_probs.append(probs.cpu().numpy())

            # 2. 파일명 처리 (확장자 제거)
            for f in file_names:
                all_files.append(f.replace('.jpg', ''))

    # 3. 모든 배치의 확률을 하나로 합침
    all_probs = np.vstack(all_probs)

    # 4. 데이터프레임 생성 (id 컬럼 + 120개 품종 컬럼)
    submission = pd.DataFrame(all_probs, columns=class_names)
    submission.insert(0, 'id', all_files) # 맨 앞에 id 컬럼 삽입

    submission.to_csv(save_path, index=False)
    print(f"제출 파일(확률 포함)이 저장되었습니다: {save_path}")

# --- 실행 부분 ---

# 1. 테스트 데이터 압축 해제 (이미 하셨다면 생략)
test_zip_path = '/content/drive/MyDrive/Colab Notebooks/Dog Breed Identification/test.zip'
with zipfile.ZipFile(test_zip_path, 'r') as z:
    z.extractall('/content/dataset/test_data')

# 2. 테스트 로더 설정 (test_transform 사용)
test_dataset = PlantTestDataset(root_dir='/content/dataset/test_data', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 3. 클래스 이름 목록 가져오기 (ImageFolder에서 썼던 순서 그대로)
class_names = full_dataset.breeds

# 4. 제출 파일 생성
generate_submission(final_model, test_loader, class_names, '/content/drive/MyDrive/Colab Notebooks/Dog Breed Identification/submission.csv')

제출 파일(확률 포함)이 저장되었습니다: /content/drive/MyDrive/Colab Notebooks/Dog Breed Identification/submission.csv
